<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [3]:
import pandas as pd
import numpy as np
import joblib
from joblib import dump, load
from sklearn.ensemble import RandomForestClassifier
import warnings
warnings.simplefilter(action='ignore')

In [12]:
class Forecast:
    """
    Предсказание рейтинга блюда или его класса
    """
    def __init__(self, list_of_ingredients):
        self.list_of_ingredients = list_of_ingredients
  
    def preprocess(self):
        """
        Этот метод преобразует список ингредиентов в структуры данных, 
        которые используются в алгоритмах машинного обучения, чтобы сделать предсказание.
        """
        df = pd.read_csv('../DS15/data/result.csv')
        df = df.drop(columns=['title','url', 'rating', 'rating_group'])
        vector = pd.DataFrame(np.zeros(df.shape), columns=df.columns)
        if set(self.list_of_ingredients).issubset(df.columns):
            vector[self.list_of_ingredients] = 1
        
        return vector
        
    def predict_rating_category(self):
        """
        Этот метод возращает рейтинговую категорию для списка ингредиентов, используя классификационную модель, 
        которая была обучена заранее. Помимо самого рейтинга, метод возвращает также и текст, 
        который дает интерпретацию этой категории и дает рекомендации, как в примере выше.
        """
        data = self.preprocess()
        model = joblib.load('../DS15/data/best_model')
        rating_cat = model.predict(data)       
        print('I. НАШ ПРОГНОЗ:\n')
        
        if rating_cat[0] == 'bad':
            return 'Невкусное. Хоть конкретно вам может быть и понравится блюдо из этих ингредиентов, но, на наш взгляд, это плохая идея – готовить блюдо из них. Хотели предупредить.'
        elif rating_cat[0] == 'so-so':
            return 'Нормальное. Но если Вы гурман, стоит поискать что-то другое.'
        elif rating_cat[0] == 'great':
            return 'Вкусное. На наш скромный взгляд из этих ингредиентов получится изысканное блюдо.'
        else:
            return 'По данным ингредиентам ничего не нашлось. Попробуйте ввести что-то другое.'



In [13]:
Forecast(['a','b', 'c', 'garlic']).predict_rating_category()

I. НАШ ПРОГНОЗ:



'Вкусное. На наш скромный взгляд из этих ингредиентов получится изысканное блюдо.'

In [18]:
class NutritionFacts:
    """
    Выдает информацию о пищевой ценности ингредиентов.
    """
    def __init__(self, list_of_ingredients):
        self.list_of_ingredients = list_of_ingredients
        self.nutrients = pd.read_csv('../DS15/data/Nutrients.csv')
        
    def retrieve(self):
        """
        Этот метод получает всю имеющуюся информацию о пищевой ценности из файла с заранее 
        собранной информацией по заданным ингредиентам. 
        Он возвращает ее в том виде, который вам кажется наиболее удобным и подходящим.
        """

        facts =pd.DataFrame(columns=self.nutrients.columns)
        
        for i in self.list_of_ingredients:
            nutr_el =self.nutrients.loc[self.nutrients.ingredient == i]
            facts = pd.concat([facts, nutr_el], ignore_index=True)          
        facts.drop(columns='Unnamed: 0',inplace=True)
        
        return facts

    def filter(self, must_nutrients, n):
        """
        Этот метод отбирает из всей информации о пищевой ценности только те нутриенты, 
        которые были заданы в must_nutrients (пример в PDF-файле ниже), 
        а также топ-n нутриентов с наибольшим значением дневной нормы потребления для заданного ингредиента. 
        Он возвращает текст, отформатированный как в примере выше.
        """
        
        facts = self.retrieve()
        info = {}
        print(f'II. ПИЩЕВАЯ ЦЕННОСТЬ \n')        
        
        for i in self.list_of_ingredients:
            per_ingred = facts.loc[facts['ingredient'] == i].sort_values('value',ascending=False)
            print(i.capitalize())
                
            for j in range(len(must_nutrients)):
                if j < n:
                    if must_nutrients[j] in per_ingred['nutrientName'].tolist():
                        nutrients = per_ingred.loc[per_ingred['nutrientName']== must_nutrients[j]]
                        value =per_ingred.loc[per_ingred['nutrientName']== must_nutrients[j]]['value']
                        print(f'{must_nutrients[j]} - {float(value)} % of daily value')
                    else:
                        print(f'Нутриент {must_nutrients[j]} для данного ингредиента не найден')
                else:
                    break

            print('\n')

In [19]:
NutritionFacts(['almond', 'zucchini']).filter(['Protein', 'Total lipid (fat)', 'Calcium, Ca', 'Iron, Fe',
       'Magnesium, Mg', 'Phosphorus, P', 'Potassium, K', 'Sodium, Na',
       'Zinc, Zn', 'Copper, Cu', 'Selenium, Se', 'Vitamin A, RAE',
       'Vitamin E (alpha-tocopherol)', 'Vitamin D (D2 + D3)',
       'Vitamin C, total ascorbic acid', 'Thiamin', 'Riboflavin',
       'Niacin', 'Vitamin B-6', 'Vitamin B-12', 'Choline, total',
       'Vitamin K (phylloquinone)', 'Cholesterol',
       'Fatty acids, total saturated', 'Pantothenic acid',
       'Manganese, Mn', 'Biotin'], 5)


II. ПИЩЕВАЯ ЦЕННОСТЬ 

Almond
Protein - 14.0 % of daily value
Total lipid (fat) - 31.0 % of daily value
Calcium, Ca - 27.0 % of daily value
Iron, Fe - 19.0 % of daily value
Magnesium, Mg - 66.0 % of daily value


Zucchini
Protein - 1.0 % of daily value
Total lipid (fat) - 0.0 % of daily value
Calcium, Ca - 2.0 % of daily value
Iron, Fe - 2.0 % of daily value
Нутриент Magnesium, Mg для данного ингредиента не найден




In [20]:
class SimilarRecipes:
    """
    Рекомендация похожих рецептов с дополнительной информацией
    """
    def __init__(self, list_of_ingredients):
        self.list_of_ingredients = list_of_ingredients
        self.recipes = pd.read_csv('../DS15/data/result.csv')
        self.ingredients = list(self.recipes.drop(columns=['title', 'url','rating','rating_group']).columns)
        
    def find_all(self):
        """
        Этот метод возвращает список индексов рецептов, которые содержат заданный список ингредиентов. 
        Если нет ни одного рецепта, содержащего все эти ингредиенты, 
        то сделайте обработку ошибки, чтобы программа не ломалась.
        """

        if set(self.list_of_ingredients).issubset(self.recipes.columns):
            recipes = self.recipes[self.list_of_ingredients]
            results = recipes.sum(axis=1) == len(self.list_of_ingredients)
            indexes = recipes.loc[results].index.tolist()
            return indexes
        else:
            indexes = []
            return indexes

    
    def top_similar(self, n):
        """
        Этот метод возвращает текст, форматированный как в примере выше: с заголовком, рейтингом и URL. 
        Чтобы это сделать, он вначале находит топ-n наиболее похожих рецептов 
        с точки зрения количества дополнительных ингредиентов, 
        которые потребуются в этих рецептах. Наиболее похожим будет тот, в котором не требуется никаких других ингредиентов. 
        Далее идет тот, у которого появляется 1 доп. ингредиент. Далее – 2. 
        Если рецепт нуждается в более, чем 5 доп. ингредиентах, то такой рецепт не выводится.
        """
        print(f'III. ТОП-{n} ПОХОДИХ РЕЦЕПТА \n')
        
        indexes = self.find_all()
        
        if len(indexes) != 0:
            final_results = []

            for i in indexes:
                if self.recipes.iloc[i].loc[self.ingredients].sum() - len(self.list_of_ingredients) < 5:
                    if self.recipes.iloc[i].loc[self.ingredients].sum() == len(self.list_of_ingredients):
                        final_results.append({'name':self.recipes.iloc[i].title, 
                                              'rating': self.recipes.iloc[i].rating, 
                                              'url': self.recipes.iloc[i].url})
                    elif self.recipes.iloc[i].loc[self.ingredients].sum() - 1 == len(self.list_of_ingredients):
                        final_results.append({'name':self.recipes.iloc[i].title, 
                                              'rating': self.recipes.iloc[i].rating, 
                                              'url': self.recipes.iloc[i].url})
                    elif self.recipes.iloc[i].loc[self.ingredients].sum() - 2 == len(self.list_of_ingredients):
                        final_results.append({'name':self.recipes.iloc[i].title, 
                                              'rating': self.recipes.iloc[i].rating, 
                                              'url': self.recipes.iloc[i].url})
                    elif self.recipes.iloc[i].loc[self.ingredients].sum() - 3 == len(self.list_of_ingredients):
                        final_results.append({'name':self.recipes.iloc[i].title, 
                                              'rating': self.recipes.iloc[i].rating, 
                                              'url': self.recipes.iloc[i].url})
                    elif self.recipes.iloc[i].loc[self.ingredients].sum() - 4 == len(self.list_of_ingredients):
                        final_results.append({'name':self.recipes.iloc[i].title, 
                                              'rating': self.recipes.iloc[i].rating, 
                                              'url': self.recipes.iloc[i].url})
            for j in final_results[:n]:
                print(j.get('name'), j.get('rating'), j.get('url'))

        else:
            print('Похожих рецептов не найдено.\n')
                


In [21]:
SimilarRecipes(['gin', 'chile pepper']).top_similar(3)

III. ТОП-3 ПОХОДИХ РЕЦЕПТА 

Похожих рецептов не найдено.



In [102]:
recipes.iloc[915].loc[ingredients].sum()

4.0

In [152]:
list_of_ingredients = ['zucchini', 'beef']

In [153]:
indexes = SimilarRecipes(list_of_ingredients).find_all()
indexes

[2883, 4391, 7337]

In [154]:
recipes = pd.read_csv('result.csv')

In [155]:
ingredients = list(recipes.drop(columns=['title', 'url', 'rating','rating_group']).columns)

In [156]:
final_results = []

for i in indexes:
    if recipes.iloc[i].loc[ingredients].sum() - len(list_of_ingredients) < 5:
        if recipes.iloc[i].loc[ingredients].sum() == len(list_of_ingredients):
            final_results.append({'name':recipes.iloc[i].title, 
                                  'rating': recipes.iloc[i].rating, 
                                  'url': ''})
        elif recipes.iloc[i].loc[ingredients].sum() - 1 == len(list_of_ingredients):
            final_results.append({'name':recipes.iloc[i].title, 
                                  'rating': recipes.iloc[i].rating, 
                                  'url': ''})
        elif recipes.iloc[i].loc[ingredients].sum() - 2 == len(list_of_ingredients):
            final_results.append({'name':recipes.iloc[i].title, 
                                  'rating': recipes.iloc[i].rating, 
                                  'url': ''})
        elif recipes.iloc[i].loc[ingredients].sum() - 3 == len(list_of_ingredients):
            final_results.append({'name':recipes.iloc[i].title, 
                                  'rating': recipes.iloc[i].rating, 
                                  'url': ''})
        elif recipes.iloc[i].loc[ingredients].sum() - 4 == len(list_of_ingredients):
            final_results.append({'name':recipes.iloc[i].title, 
                                  'rating': recipes.iloc[i].rating, 
                                  'url': ''})
for j in final_results[:5]:
    print(j.get('name'), j.get('rating'), j.get('url'))

Skirt Steak with Poblano Rajas and Zucchini  3.125 


In [19]:
list_of_ingredients = input('Введите не менее двух ингредиентов на английском языке через запятую:')
list_of_ingredients= list_of_ingredients.lower().split(',')
list_of_ingredients = [x.strip(' ') for x in list_of_ingredients]

Введите не менее двух ингредиентов на английском языке через запятую:patato, beef


In [20]:
list_of_ingredients

['patato', 'beef']

In [22]:
class Recommend_Menu():
    
    def __init__(self):
        self.data= pd.read_csv('../DS15/data/result.csv')
        self.full_recipes = pd.read_json('full_format_recipes.json', orient='columns')
        self.full_recipes = self.full_recipes[self.full_recipes['categories'].notna()]
        self.full_recipes = self.full_recipes.loc[self.full_recipes['title'].isin(list(self.data['title']))]
        self.meal = [['Breakfast','ЗАВТРАК'], ['Lunch','ОБЕД'], ['Dinner','УЖИН'], ['Dessert','ДЕСЕРТ']]
        
    def recommend(self):

        for i in self.meal:
            print(f'{i[1]}\n')
            info = self.full_recipes[self.full_recipes.categories.map(set([i[0]]).issubset)].sample(n=1)

            name = info.title.tolist()[0]

            info_url = self.data.loc[self.data.title ==name]
            print(F'{name} (рейтинг: {info.rating.tolist()[0]})\n')
            print('Ингредиенты: \n')
            info_ings = info_url.drop(columns=['title', 'url', 'rating','rating_group']).T
            info_ings = info_ings.reset_index().rename(columns={'index':'ings'})
            
            ings = info_ings.loc[info_ings[info_ings.columns[1]] == 1].index.tolist()
            ings = info_ings.iloc[ings]['ings'].tolist()

            for j in ings:
                print(f'-{j}')
            print('\n') 
            print('Nutrients: \n')
            print(f'- calories: {round(float(info.calories)/2500* 100, 2)} %\n- protein: {round(float(info.protein) / 150 * 100, 2)} %\
            \n- fat: {round(float(info.fat) / 178 * 100.2)} %\n- sodium:{round(float(info.sodium) / 2300 * 100,2)} %\n')
            print(f'Ссылка на рецепт: {info_url.url.tolist()[0]}\n')

In [23]:
Recommend_Menu().recommend()

ЗАВТРАК

Black-Eyed Pea Fritters with Hot Pepper Relish  (рейтинг: 4.375)

Ингредиенты: 

-bean
-bell pepper
-hot pepper
-party
-pea
-pepper


Nutrients: 

- calories: 2.76 %
- protein: 0.67 %            
- fat: 3 %
- sodium:7.17 %

Ссылка на рецепт: https://www.epicurious.com/recipes/food/views/black-eyed-pea-fritters-with-hot-pepper-relish-106044

ОБЕД

Smorgastarta  (рейтинг: 5.0)

Ингредиенты: 

-chill
-cream cheese
-egg
-herb
-salmon


Nutrients: 

- calories: 19.52 %
- protein: 11.33 %            
- fat: 20 %
- sodium:28.57 %

Ссылка на рецепт: https://www.epicurious.com/recipes/food/views/smorgastarta-14043

УЖИН

Veal Tournedos in Cream Sauce (Veau à la Crème)  (рейтинг: 4.375)

Ингредиенты: 

-ham
-swiss cheese
-veal


Nutrients: 

- calories: 28.52 %
- protein: 40.0 %            
- fat: 29 %
- sodium:42.22 %

Ссылка на рецепт: https://google.com/search?q=Veal%20Tournedos%20in%20Cream%20Sauce%20(Veau%20%20la%20Crme)+recipe

ДЕСЕРТ

Hazelnut Brittle  (рейтинг: 3.125)

Ингредиен

In [432]:
data.loc[data.title=='Steamed Fish with Scallions and Ginger']

,title,url,rating,almond,amaretto,anchovy,anise,apple,apple juice,apricot,...,white wine,whole wheat,wild rice,wine,wok,yellow squash,yogurt,yuca,zucchini,rating_group


In [435]:
full_recipes.loc[full_recipes['title'].isin(list(data['title']))]

,directions,fat,date,categories,calories,desc,protein,rating,title,ingredients,sodium
0,"[1. Place the stock, lentils, celery, carrot, ...",7.0,2006-09-01 04:00:00+00:00,"[Sandwich, Bean, Fruit, Tomato, turkey, Vegeta...",426.0,None,30.0,2.500,"Lentil, Apple, and Turkey Wrap","[4 cups low-sodium vegetable or chicken stock,...",559.0
1,[Combine first 9 ingredients in heavy medium s...,23.0,2004-08-20 04:00:00+00:00,"[Food Processor, Onion, Pork, Bake, Bastille D...",403.0,This uses the same ingredients found in boudin...,18.0,4.375,Boudin Blanc Terrine with Red Onion Confit,"[1 1/2 cups whipping cream, 2 medium onions, c...",1439.0
2,[In a large heavy saucepan cook diced fennel a...,7.0,2004-08-20 04:00:00+00:00,"[Soup/Stew, Dairy, Potato, Vegetable, Fennel, ...",165.0,None,6.0,3.750,Potato and Fennel Soup Hodge,"[1 fennel bulb (sometimes called anise), stalk...",165.0
4,[Preheat oven to 350°F. Lightly grease 8x8x2-i...,32.0,2004-08-20 04:00:00+00:00,"[Cheese, Dairy, Pasta, Vegetable, Side, Bake, ...",547.0,None,20.0,3.125,Spinach Noodle Casserole,"[1 12-ounce package frozen spinach soufflé, th...",452.0
5,"[Mix basil, mayonnaise and butter in processor...",79.0,2004-08-20 04:00:00+00:00,"[Sandwich, Food Processor, Tomato, Kid-Friendl...",948.0,This recipe can be prepared in 45 minutes or l...,19.0,4.375,The Best Blts,[2 1/2 cups (lightly packed) fresh basil leave...,1042.0
...,...,...,...,...,...,...,...,...,...,...,...
20125,[Beat whites in a bowl with an electric mixer ...,2.0,2004-08-20 04:00:00+00:00,"[Mixer, Cheese, Egg, Fry, Cocktail Party, Parm...",28.0,None,2.0,3.125,Parmesan Puffs,"[2 large egg whites, 3 oz Parmigiano-Reggiano,...",64.0
20126,[Bring broth to simmer in saucepan.Remove from...,28.0,2008-02-28 22:06:54+00:00,"[Side, Kid-Friendly, High Fiber, Dinner, Parme...",671.0,Cooking the artichokes with the rice infuses t...,22.0,4.375,Artichoke and Parmesan Risotto,"[5 1/2 cups (or more) low-salt chicken broth, ...",583.0
20127,"[Using a sharp knife, cut a shallow X in botto...",38.0,2005-10-21 18:21:20+00:00,"[Onion, Poultry, turkey, Vegetable, Bake, Kid-...",563.0,None,31.0,4.375,Turkey Cream Puff Pie,"[1 small tomato, 1 small onion, finely chopped...",652.0
20128,[Heat 2 tablespoons oil in heavy medium skille...,24.0,2004-08-20 04:00:00+00:00,"[Milk/Cream, Citrus, Dairy, Fish, Garlic, Past...",631.0,"Sharon Hooykaas of Los Alamitos, California, w...",45.0,4.375,Snapper on Angel Hair with Citrus Cream,"[4 tablespoons olive oil, 4 shallots, thinly s...",517.0


In [219]:
data= pd.read_csv('result.csv')
full_recipes = pd.read_json('full_format_recipes.json', orient='columns')
full_recipes = full_recipes[full_recipes['categories'].notna()]

In [385]:
breakfast = full_recipes[full_recipes.categories.map(set(['Breakfast']).issubset)].sample(n=1)
lunch = full_recipes[full_recipes.categories.map(set(['Lunch']).issubset)].sample(n=1)
dinner = full_recipes[full_recipes.categories.map(set(['Dinner']).issubset)].sample(n=1)
dessert = full_recipes[full_recipes.categories.map(set(['Dessert']).issubset)].sample(n=1)

In [386]:
breakfast

,directions,fat,date,categories,calories,desc,protein,rating,title,ingredients,sodium
12147,"[Preheat oven to 350°F., In a large bowl, comb...",30.0,2014-08-12 04:00:00+00:00,"[Breakfast, Cherry, Coconut, Oat, Honey, Granola]",530.0,None,10.0,5.0,Cherry and Coconut Granola with Yogurt,"[3 cups rolled oats, 1 cup chopped almonds, 3/...",201.0


In [387]:
breakfast_name = breakfast.title.tolist()[0]
breakfast_name

'Cherry and Coconut Granola with Yogurt '

In [427]:
# if breakfast_name in data.title:
info_url = data.loc[data.title ==breakfast_name]

In [428]:
info_ings = info_url.drop(columns=['title', 'url', 'rating','rating_group']).T
info_ings = info_ings.reset_index().rename(columns={'index':'ings'})

ings = info_ings.loc[info_ings[info_ings.columns[1]] == 1].index.tolist()
ings = info_ings.iloc[ings]['ings'].tolist()

In [429]:
ings

['cherry', 'coconut', 'honey', 'oat']

In [393]:
breakfast_2=breakfast_2.drop(columns=['title', 'url', 'rating','rating_group']).T

In [417]:
breakfast_2 = breakfast_2.reset_index().rename(columns={'index':'ings'})

In [420]:
ings = breakfast_2.loc[breakfast_2[breakfast_2.columns[1]] == 1].index.tolist()

In [424]:
breakfast_2.iloc[ings]['ings'].tolist()

['cherry', 'coconut', 'honey', 'oat']

In [376]:
meal = [['Breakfast','ЗАВТРАК'], ['Lunch','ОБЕД'], ['Dinner','УЖИН'], ['Dessert','ДЕСЕРТ']]

for i in meal:
    print(f'{i[1]}\n')
    info = full_recipes[full_recipes.categories.map(set([i[0]]).issubset)].sample(n=1)
    
    name = info.title.tolist()[0]

    info_url = data.loc[data.title ==name]
    print(F'{name} (рейтинг: {info.rating.tolist()[0]})\n')
    print('Ингредиенты: \n')
    info_ings = info_url.drop(columns=['title', 'url', 'rating','rating_group']).T
    ings = info_ings .loc[info_ings[info_ings.columns[0]] == 1].index.tolist()
    
    for j in ings:
        print(f'-{j}')
    print('\n')

    print('Nutrients: \n')
    print(f'- calories: {round(float(info.calories)/2500* 100, 2)} %\n- protein: {round(float(info.protein) / 150 * 100, 2)} %\
    \n- fat: {round(float(info.fat) / 178 * 100.2)} %\n- sodium:{round(float(info.sodium) / 2300 * 100,2)} %\n')
    print(f'Ссылка на рецепт: {info_url.url.tolist()[0]}\n')


ЗАВТРАК

Cranberry-Orange Scones  (рейтинг: 5.0)

Ингредиенты: 

-bread
-citrus
-cranberry
-orange


Nutrients: 

- calories: 16.16 %
- protein: 4.0 %    
- fat: 10 %
- sodium:23.7 %

Ссылка на рецепт: https://www.epicurious.com/recipes/food/views/cranberry-orange-scones-15682

ОБЕД

Butternut Squash Soup with Pumpkin Butter  (рейтинг: 3.75)

Ингредиенты: 

-butternut squash
-pistachio
-pumpkin
-squash


Nutrients: 

- calories: 20.68 %
- protein: 3.33 %    
- fat: 27 %
- sodium:2.57 %

Ссылка на рецепт: https://www.epicurious.com/recipes/food/views/butternut-squash-soup-with-pumpkin-butter-51122000

УЖИН

Orecchiette with Fresh Mozzarella, Grape Tomatoes, and Garlic Chives  (рейтинг: 2.5)

Ингредиенты: 

-cheese
-garlic
-mozzarella
-party
-pasta
-tomato
-vegetable


Nutrients: 

- calories: 32.72 %
- protein: 24.0 %    
- fat: 26 %
- sodium:31.52 %

Ссылка на рецепт: https://www.epicurious.com/recipes/food/views/orecchiette-with-fresh-mozzarella-grape-tomatoes-and-garlic-chives-354335

In [369]:
print('Nutrients: \n')
print(f'- {round(float(breakfast.calories)/2500* 100, 2)} %\n- {round(float(breakfast.protein) / 150 * 100, 2)} %\
\n- {round(float(breakfast.fat) / 178 * 100.2)} %\n- {round(float(breakfast.sodium) / 2300 * 100,2)} %')


Nutrients: 

- 12.2 %
- 2.67 %
- 8 %
- 8.26 %


In [25]:
list_of_ingredients = input('Введите не менее двух ингредиентов на английском языке через запятую:')
list_of_ingredients= list_of_ingredients.lower().split(',')
list_of_ingredients = [i.strip(' ') for i in list_of_ingredients]

Введите не менее двух ингредиентов на английском языке через запятую:bread, avocado, cheese, garlic


In [27]:
df = pd.read_csv('../DS15/data/result.csv')
ing  = df.drop(columns=['title','url', 'rating', 'rating_group'])


if len(list_of_ingredients) < 2:
    print('Пожалуйста, введите не менее двух ингредиентов')
else:
    a = 1
    for i in list_of_ingredients:
        if i not in list(ing):
            print('Данный ингредиент не найден:', i)
            list_of_ingredients.remove(i)
        else:
            a += 1
    if a < len(list_of_ingredients):
        print('Добавьте новый ингредиент')
      

In [29]:
print(Forecast(list_of_ingredients).predict_rating_category())
print('\n')
print(NutritionFacts(list_of_ingredients).filter(['Protein', 'Total lipid (fat)', 'Calcium, Ca', 'Sodium, Na','Vitamin C','Cholesterol'],6))
print(SimilarRecipes(list_of_ingredients).top_similar(3))


I. НАШ ПРОГНОЗ:

Вкусное. На наш скромный взгляд из этих ингредиентов получится изысканное блюдо.


II. ПИЩЕВАЯ ЦЕННОСТЬ 

Bread
Protein - 5.0 % of daily value
Total lipid (fat) - 0.0 % of daily value
Calcium, Ca - 0.0 % of daily value
Sodium, Na - 24.0 % of daily value
Нутриент Vitamin C для данного ингредиента не найден
Cholesterol - 0.0 % of daily value


Avocado
Protein - 1.0 % of daily value
Total lipid (fat) - 24.0 % of daily value
Calcium, Ca - 3.0 % of daily value
Sodium, Na - 38.0 % of daily value
Нутриент Vitamin C для данного ингредиента не найден
Cholesterol - 13.0 % of daily value


Cheese
Protein - 14.0 % of daily value
Total lipid (fat) - 14.0 % of daily value
Calcium, Ca - 0.0 % of daily value
Sodium, Na - 26.0 % of daily value
Нутриент Vitamin C для данного ингредиента не найден
Cholesterol - 18.0 % of daily value


Garlic
Protein - 0.0 % of daily value
Total lipid (fat) - 0.0 % of daily value
Calcium, Ca - 51.0 % of daily value
Sodium, Na - 0.0 % of daily value
Нутрие

In [30]:
print(Recommend_Menu().recommend())

ЗАВТРАК

Pancakes with Warm Maple Syrup & Coffee Butter  (рейтинг: 3.75)

Ингредиенты: 

-butter
-coffee


Nutrients: 

- calories: 9.6 %
- protein: 4.0 %            
- fat: 8 %
- sodium:16.43 %

Ссылка на рецепт: https://www.epicurious.com/recipes/food/views/pancakes-with-warm-maple-syrup-coffee-butter-51190420

ОБЕД

Cavatelli Salad with Artichokes and Goat Cheese  (рейтинг: 3.75)

Ингредиенты: 

-artichoke
-basil
-chill
-goat cheese
-pasta
-tomato


Nutrients: 

- calories: 19.48 %
- protein: 9.33 %            
- fat: 15 %
- sodium:6.43 %

Ссылка на рецепт: https://www.epicurious.com/recipes/food/views/cavatelli-salad-with-artichokes-and-goat-cheese-106939

УЖИН

Sweet-and-Sour Balsamic-Glazed Spareribs  (рейтинг: 4.375)

Ингредиенты: 

-meat
-pork
-pork rib
-vinegar


Nutrients: 

- calories: 93.64 %
- protein: 69.33 %            
- fat: 86 %
- sodium:82.35 %

Ссылка на рецепт: https://www.epicurious.com/recipes/food/views/sweet-and-sour-balsamic-glazed-spareribs-51129400

ДЕСЕРТ

